#12. Suffixes

Suffixes provide a mechanism for declaring extraneous model data, which can be used in a number of contexts. Most commonly, suffixes are used by solver plugins to store extra information about the solution of a model. This and other suffix functionality is made available to the modeler through the use of the Suffix component class. Uses of Suffix include:

- Importing extra information from a solver about the solution of a mathematical program (e.g., constraint duals, variable reduced costs, basis information).

- Exporting information to a solver or algorithm to aid in solving a mathematical program (e.g., warm-starting information, variable branching priorities).

- Tagging modeling components with local data for later use in advanced scripting algorithms.

###12.1. Suffix Notation and the Pyomo NL File Interface
The Suffix component used in Pyomo has been adapted from the suffix notation used in the modeling language AMPL [AMPL](https://software.sandia.gov/downloads/pub/pyomo/PyomoOnlineDocs.html#AMPL). Therefore, it follows naturally that AMPL style suffix functionality is fully available using Pyomo’s NL file interface. For information on AMPL style suffixes the reader is referred to the AMPL website:

http://www.ampl.com

A number of scripting examples that highlight the use AMPL style suffix functionality are available in the <span style="color:darkblue; font-family:Courier">examples/pyomo/suffixes</span> directory distributed with Pyomo.

###12.2. Declaration
The effects of declaring a Suffix component on a Pyomo model are determined by the following traits:

- direction: This trait defines the direction of information flow for the suffix. A suffix direction can be assigned one of four possible values:

    - <span style="color:darkblue; font-family:Courier">LOCAL</span> - suffix data stays local to the modeling framework and will not be imported or exported by a solver plugin (default)

    - <span style="color:darkblue; font-family:Courier">IMPORT</span> - suffix data will be imported from the solver by its respective solver plugin

    - <span style="color:darkblue; font-family:Courier">EXPORT</span> - suffix data will be exported to a solver by its respective solver plugin

    - <span style="color:darkblue; font-family:Courier">IMPORT_EXPORT</span> - suffix data flows in both directions between the model and the solver or algorithm

- datatype: This trait advertises the type of data held on the suffix for those interfaces where it matters (e.g., the NL file interface). A suffix datatype can be assigned one of three possible values:

    - <span style="color:darkblue; font-family:Courier">FLOAT</span> - the suffix stores floating point data (default)

    - <span style="color:darkblue; font-family:Courier">INT</span> - the suffix stores integer data

    - <span style="color:darkblue; font-family:Courier">None</span> - the suffix stores any type of data

>#####Note
>Exporting suffix data through Pyomo’s NL file interface requires all active export suffixes have a strict datatype (i.e., <span style="color:darkblue; font-family:Courier">datatype=None</span> is not allowed).

The following code snippet shows examples of declaring a Suffix component on a Pyomo model:



In [1]:
from pyomo.environ import *

model = ConcreteModel()

# Export integer data
model.priority = Suffix(direction=Suffix.EXPORT, datatype=Suffix.INT)

# Export and import floating point data
model.dual = Suffix(direction=Suffix.IMPORT_EXPORT)

# Store floating point data
model.junk = Suffix()

Declaring a Suffix with a non-local direction on a model is not guaranteed to be compatible with all solver plugins in Pyomo. Whether a given Suffix is acceptable or not depends on both the solver and solver interface being used. In some cases, a solver plugin will raise an exception if it encounters a Suffix type that it does not handle, but this is not true in every situation. For instance, the NL file interface is generic to all AMPL-compatible solvers, so there is no way to validate that a Suffix of a given name, direction, and datatype is appropriate for a solver. One should be careful in verifying that Suffix declarations are being handled as expected when switching to a different solver or solver interface.

###12.3. Operations
The Suffix component class provides a dictionary interface for mapping Pyomo modeling components to arbitrary data. This mapping functionality is captured within the ComponentMap base class, which is also available within Pyomo’s modeling environment. The ComponentMap can be used as a more lightweight replacement for Suffix in cases where a simple mapping from Pyomo modeling components to arbitrary data values is required.

>#####Note
>ComponentMap and Suffix use the built-in <span style="color:darkblue; font-family:Courier">id()</span> function for hashing entry keys. This design decision arises from the fact that most of the modeling components found in Pyomo are either not hashable or use a hash based on a mutable numeric value, making them unacceptable for use as keys with the built-in dict class.


>#####Warning
>The use of the built-in <span style="color:darkblue; font-family:Courier">id()</span> function for hashing entry keys in ComponentMap and Suffix makes them inappropriate for use in situations where built-in object types must be used as keys. It is strongly recommended that only Pyomo modeling components be used as keys in these mapping containers (<span style="color:darkblue; font-family:Courier">Var</span>, <span style="color:darkblue; font-family:Courier">Constraint</span>, etc.).



>#####Warning
>Do not attempt to pickle or deepcopy instances of ComponentMap or Suffix unless doing so along with the components for which they hold mapping entries. As an example, placing one of these objects on a model and then cloning or pickling that model is an acceptable scenario.

In addition to the dictionary interface provided through the ComponentMap base class, the Suffix component class also provides a number of methods whose default semantics are more convenient for working with indexed modeling components. The easiest way to highlight this functionality is through the use of an example.



In [2]:
from pyomo.environ import *

model = ConcreteModel()
model.x = Var()
model.y = Var([1,2,3])
model.foo = Suffix()

In this example we have a concrete Pyomo model with two different types of variable components (indexed and non-indexed) as well as a Suffix declaration (foo). The next code snippet shows examples of adding entries to the suffix foo.

In [3]:
# Assign a suffix value of 1.0 to model.x
model.foo.setValue(model.x, 1.0)

# Same as above with dict interface
model.foo[model.x] = 1.0


# Assign a suffix value of 0.0 to all indices of model.y
# By default this expands so that entries are created for
# every index (y[1], y[2], y[3]) and not model.y itself
model.foo.setValue(model.y, 0.0)

# The same operation using the dict interface results in an entry only
# for the parent component model.y
model.foo[model.y] = 50.0


# Assign a suffix value of -1.0 to model.y[1]
model.foo.setValue(model.y[1], -1.0)

# Same as above with the dict interface
model.foo[model.y[1]] = -1.0

In this example we highlight the fact that the <span style="color:darkblue; font-family:Courier">setitem</span> and <span style="color:darkblue; font-family:Courier">setValue</span> entry methods can be used interchangeably except in the case where indexed components are used (model.y). In the indexed case, the <span style="color:darkblue; font-family:Courier">setitem</span> approach creates a single entry for the parent indexed component itself, whereas the <span style="color:darkblue; font-family:Courier">setValue</span> approach by default creates an entry for each index of the component. This behavior can be controlled using the optional keyword <span style="color:darkblue">*expand*</span>, where assigning it a value of <span style="color:darkblue; font-family:Courier">False</span> results in the same behavior as <span style="color:darkblue; font-family:Courier">setitem</span>.

Other operations like accessing or removing entries in our mapping can performed as if the built-in <span style="color:darkblue; font-family:Courier">dict</span> class is in use.



In [4]:
print(model.foo.get(model.x))         # -> 1.0
print(model.foo[model.x])             # -> 1.0

print(model.foo.get(model.y[1]))      # -> -1.0
print(model.foo[model.y[1]])          # -> -1.0

print(model.foo.get(model.y[2]))      # -> 0.0
print(model.foo[model.y[2]])          # -> 0.0

print(model.foo.get(model.y))         # -> 50.0
print(model.foo[model.y])             # -> 50.0

del model.foo[model.y]

print(model.foo.get(model.y))         # -> None
print(model.foo[model.y])             # -> raise KeyError

1.0
1.0
-1.0
-1.0
0.0
0.0
50.0
50.0
None


KeyError: 'Component with name: y (id=4535312960)'

The non-dict method <span style="color:darkblue; font-family:Courier">clearValue</span> can be used in place of <span style="color:darkblue; font-family:Courier">delitem</span> to remove entries, where it inherits the same default behavior as <span style="color:darkblue; font-family:Courier">setValue</span> for indexed components and does not raise a KeyError when the argument does not exist as a key in the mapping.



In [5]:
model.foo.clearValue(model.y)

print(model.foo[model.y[1]])          # -> raise KeyError

del model.foo[model.y[1]]             # -> raise KeyError

model.foo.clearValue(model.y[1])      # -> does nothing

KeyError: 'Component with name: y[1] (id=4558940576)'

A summary non-dict Suffix methods is provided here:


###12.4. Importing Suffix Data
Importing suffix information from a solver solution is achieved by declaring a Suffix component with the appropriate name and direction. Suffix names available for import may be specific to third-party solvers as well as individual solver interfaces within Pyomo. The most common of these, available with most solvers and solver interfaces, is constraint dual multipliers. Requesting that duals be imported into suffix data can be accomplished by declaring a Suffix component on the model.



In [6]:
from pyomo.environ import *

model = ConcreteModel()
model.dual = Suffix(direction=Suffix.IMPORT)
model.x = Var()
model.obj = Objective(expr=model.x)
model.con = Constraint(expr=model.x>=1.0)

The existence of an active suffix with the name dual that has an import style suffix direction will cause constraint dual information to be collected into the solver results (assuming the solver supplies dual information). In addition to this, after loading solver results into a problem instance (using a python script or Pyomo callback functions in conjunction with the <span style="color:darkblue; font-family:Courier">pyomo</span> command), one can access the dual values associated with constraints using the dual Suffix component.

In [7]:
print(instance.dual[instance.con]) # -> 1.0

NameError: name 'instance' is not defined

Alternatively, the <span style="color:darkblue; font-family:Courier">pyomo</span> option <span style="color:darkblue; font-family:Courier">--solver-suffixes</span> can be used to request suffix information from a solver. In the event that suffix names are provided via this command-line option, the <span style="color:darkblue; font-family:Courier">pyomo</span> script will automatically declare these Suffix components on the constructed instance making these suffixes available for import.

###12.5. Exporting Suffix Data
Exporting suffix data is accomplished in a similar manner as to that of importing suffix data. One simply needs to declare a Suffix component on the model with an export style suffix direction and associate modeling component values with it. The following example shows how one can declare a special ordered set of type 1 using AMPL-style suffix notation in conjunction with Pyomo’s NL file interface.



In [8]:
from pyomo.environ import *

model = ConcreteModel()
model.y = Var([1,2,3],within=NonNegativeReals)

model.sosno = Suffix(direction=Suffix.EXPORT)
model.ref = Suffix(direction=Suffix.EXPORT)

# Add entry for each index of model.y
model.sosno.setValue(model.y,1)
model.ref[model.y[1]] = 0
model.ref[model.y[2]] = 1
model.ref[model.y[3]] = 2

Most AMPL-compatible solvers will recognize the suffix names <span style="color:darkblue; font-family:Courier">sosno</span> and <span style="color:darkblue; font-family:Courier">ref</span> as declaring a special ordered set, where a positive value for <span style="color:darkblue; font-family:Courier">sosno</span> indicates a special ordered set of type 1 and a negative value indicates a special ordered set of type 2.

>#####Note
>Pyomo provides the SOSConstraint component for declaring special ordered sets, which is recognized by all solver interface, including the NL file interface.

Pyomo’s NL file interface will recognize an EXPORT style Suffix component with the name <span style="color:darkblue; font-family:Courier">dual</span> as supplying initializations for constraint multipliers. As such it will be treated separately than all other EXPORT style suffixes encountered in the NL writer, which are treated as AMPL-style suffixes. The following example script shows how one can warmstart the interior-point solver Ipopt by supplying both primal (variable values) and dual (suffixes) solution information. This dual suffix information can be both imported and exported using a single Suffix component with an IMPORT_EXPORT direction.

In [10]:
from pyomo.environ import *
from pyomo.opt import SolverFactory

### Create the ipopt solver plugin using the ASL interface
solver = 'ipopt'
solver_io = 'nl'
stream_solver = True     # True prints solver output to screen
keepfiles =     False    # True prints intermediate file names (.nl,.sol,...)
opt = SolverFactory(solver,solver_io=solver_io)

if opt is None:
    print("")
    print("ERROR: Unable to create solver plugin for %s "\
          "using the %s interface" % (solver, solver_io))
    print("")
    exit(1)
###

### Create the example model
model = ConcreteModel()
model.x1 = Var(bounds=(1,5),initialize=1.0)
model.x2 = Var(bounds=(1,5),initialize=5.0)
model.x3 = Var(bounds=(1,5),initialize=5.0)
model.x4 = Var(bounds=(1,5),initialize=1.0)
model.obj = Objective(expr=model.x1*model.x4*(model.x1+model.x2+model.x3) + model.x3)
model.inequality = Constraint(expr=model.x1*model.x2*model.x3*model.x4 >= 25.0)
model.equality = Constraint(expr=model.x1**2 + model.x2**2 + model.x3**2 + model.x4**2 == 40.0)
###

### Declare all suffixes
# Ipopt bound multipliers (obtained from solution)
model.ipopt_zL_out = Suffix(direction=Suffix.IMPORT)
model.ipopt_zU_out = Suffix(direction=Suffix.IMPORT)
# Ipopt bound multipliers (sent to solver)
model.ipopt_zL_in = Suffix(direction=Suffix.EXPORT)
model.ipopt_zU_in = Suffix(direction=Suffix.EXPORT)
# Obtain dual solutions from first solve and send to warm start
model.dual = Suffix(direction=Suffix.IMPORT_EXPORT)
###

### Send the model to ipopt and collect the solution
print("")
print("INITIAL SOLVE")
results = opt.solve(model,keepfiles=keepfiles,tee=stream_solver)
# load the results (including any values for previously declared
# IMPORT / IMPORT_EXPORT Suffix components)
model.solutions.load_from(results)
###

### Set Ipopt options for warm-start
# The current values on the ipopt_zU_out and
# ipopt_zL_out suffixes will be used as initial
# conditions for the bound multipliers to solve
# the new problem
model.ipopt_zL_in.update(model.ipopt_zL_out)
model.ipopt_zU_in.update(model.ipopt_zU_out)
opt.options['warm_start_init_point'] = 'yes'
opt.options['warm_start_bound_push'] = 1e-6
opt.options['warm_start_mult_bound_push'] = 1e-6
opt.options['mu_init'] = 1e-6
###

### Send the model and suffix information to ipopt and collect the solution
print("")
print("WARM-STARTED SOLVE")
# The solver plugin will scan the model for all active suffixes
# valid for importing, which it will store into the results object
results = opt.solve(model,keepfiles=keepfiles,tee=stream_solver)
# load the results (including any values for previously declared
# IMPORT / IMPORT_EXPORT Suffix components)
model.solutions.load_from(results)
###


INITIAL SOLVE


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.3, running with linear solver ma27.

Number of nonzeros in equality constraint Jacobian...:        4
Number of nonzeros in inequality constraint Jacobian.:        4
Number of nonzeros in Lagrangian Hessian.............:       10

Total number of variables............................:        4
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        4
                     variables with only upper bounds:        0
Total number of equality constraints.................:        1
Total number of ine

AttributeError: 'ConcreteModel' object has no attribute 'solutions'

The difference in performance can be seen by examining Ipopt’s iteration log with and without warm starting:

- Without Warmstart:

In [11]:
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  1.6109693e+01 1.12e+01 5.28e-01  -1.0 0.00e+00    -  0.00e+00 0.00e+00   0
   1  1.6982239e+01 7.30e-01 1.02e+01  -1.0 6.11e-01    -  7.19e-02 1.00e+00f  1
   2  1.7318411e+01 3.60e-02 5.05e-01  -1.0 1.61e-01    -  1.00e+00 1.00e+00h  1
   3  1.6849424e+01 2.78e-01 6.68e-02  -1.7 2.85e-01    -  7.94e-01 1.00e+00h  1
   4  1.7051199e+01 4.71e-03 2.78e-03  -1.7 6.06e-02    -  1.00e+00 1.00e+00h  1
   5  1.7011979e+01 7.19e-03 8.50e-03  -3.8 3.66e-02    -  9.45e-01 9.98e-01h  1
   6  1.7014271e+01 1.74e-05 9.78e-06  -3.8 3.33e-03    -  1.00e+00 1.00e+00h  1
   7  1.7014021e+01 1.23e-07 1.82e-07  -5.7 2.69e-04    -  1.00e+00 1.00e+00h  1
   8  1.7014017e+01 1.77e-11 2.52e-11  -8.6 3.32e-06    -  1.00e+00 1.00e+00h  1

Number of Iterations....: 8

SyntaxError: invalid syntax (<ipython-input-11-0136aa2969ee>, line 1)

- With Warmstart:

In [12]:
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  1.7014032e+01 2.00e-06 4.07e-06  -6.0 0.00e+00    -  0.00e+00 0.00e+00   0
   1  1.7014019e+01 3.65e-12 1.00e-11  -6.0 2.50e-01    -  1.00e+00 1.00e+00h  1
   2  1.7014017e+01 4.48e-12 6.43e-12  -9.0 1.92e-06    -  1.00e+00 1.00e+00h  1

Number of Iterations....: 2

SyntaxError: invalid syntax (<ipython-input-12-cf200da9f999>, line 1)

###12.6 Using Suffixes With an AbstractModel

In order to allow the declaration of suffix data within the framework of an AbstractModel, the Suffix component can be initialized with an optional construction rule. As with constraint rules, this function will be executed at the time of model construction. The following simple example highlights the use of the <span style="color:darkblue; font-family:Courier">rule</span> keyword in suffix initialization. Suffix rules are expected to return an iterable of (component, value) tuples, where the <span style="color:darkblue; font-family:Courier">expand=True</span> semantics are applied for indexed components.

 

In [13]:
from pyomo.environ import *

model = AbstractModel()
model.x = Var()
model.c = Constraint(expr= model.x >= 1)

def foo_rule(m):
   return ((m.x, 2.0), (m.c, 3.0))
model.foo = Suffix(rule=foo_rule)

# Instantiate the model
inst = model.create()
print(inst.foo[model.x]) # -> raise KeyError
print(inst.foo[inst.x])  # -> 2.0
print(inst.foo[inst.c])  # -> 3.0

AttributeError: 'AbstractModel' object has no attribute 'create'

The next example shows an abstract model where suffixes are attached only to the variables:

In [14]:
from pyomo.environ import *

model = AbstractModel()
model.I = RangeSet(1,4)
model.x = Var(model.I)
def c_rule(m, i):
    return m.x[i] >= i
model.c = Constraint(model.I, rule=c_rule)

def foo_rule(m):
    return ((m.x[i], 3.0*i) for i in m.I)
model.foo = Suffix(rule=foo_rule)

# instantiate the model
inst = model.create_instance()
for i in inst.I:
    print (i, inst.foo[inst.x[i]])

1 3.0
2 6.0
3 9.0
4 12.0
